# GrimAge

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya
import numpy as np

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.GrimAge)

class GrimAge(pyagingModel):
    def __init__(self):
        super().__init__()

        self.PACKYRS = None
        self.ADM = None
        self.B2M = None
        self.CystatinC = None
        self.GDF15 = None
        self.Leptin = None
        self.PAI1 = None
        self.TIMP1 = None

        self.features_PACKYRS = None
        self.features_ADM = None
        self.features_B2M = None
        self.features_CystatinC = None
        self.features_GDF15 = None
        self.features_Leptin = None
        self.features_PAI1 = None
        self.features_TIMP1 = None

    def forward(self, x):
        Female = x[:, -2].unsqueeze(1)
        Age = x[:, -1].unsqueeze(1)

        PACKYRS = self.PACKYRS(x[:, self.features_PACKYRS])
        ADM = self.ADM(x[:, self.features_ADM])
        B2M = self.B2M(x[:, self.features_B2M])
        CystatinC = self.CystatinC(x[:, self.features_CystatinC])
        GDF15 = self.GDF15(x[:, self.features_GDF15])
        Leptin = self.Leptin(x[:, self.features

In [3]:
model = pya.models.GrimAge()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'grimage'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2019
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Lu, Ake T., et al. \"DNA methylation GrimAge strongly predicts lifespan and healthspan.\" Aging (albany NY) 11.2 (2019): 303."
model.metadata["doi"] = "https://doi.org/10.18632/aging.101684"
model.metadata["research_only"] = True
model.metadata["notes"] = None

## Download clock dependencies

In [5]:
#download PCClock Rdata file from https://yale.app.box.com/s/kq0b0a7lxckxjvaz7x5n4keaug7tewry
logger = pya.logger.Logger()
urls = [
    "https://pyaging.s3.amazonaws.com/supporting_files/ElasticNet_DNAmProtein_Vars_model4.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/datMiniAnnotation3_Gold.csv",
]
dir = "."
for url in urls:
    pya.utils.download(url, dir, logger, indent_level=1)

|-----------> Downloading data to ./ElasticNet_DNAmProtein_Vars_model4.csv


|-----------> in progress: 20.8889%

|-----------> in progress: 41.7778%

|-----------> in progress: 62.6667%

|-----------> in progress: 83.5556%

...

|-----------> in progress: 94.6423%

|-----------> in progress: 96.3631%

|-----------> in progress: 98.0839%

|-----------> in progress: 99.8046%

|-----------> in progress: 100.0000%


## Load features

#### From CSV

In [6]:
df = pd.read_csv('ElasticNet_DNAmProtein_Vars_model4.csv')
model.features = np.unique(df['var']).tolist()[2:] + ['female'] + ['age']

## Load weights into base model

#### Linear model

In [7]:
all_features = np.unique(df['var']).tolist()[2:] + ['Female'] + ['Age']

model.PACKYRS = pya.models.LinearModel(input_dim=len(np.array(df.loc[df['Y.pred'] == 'DNAmPACKYRS'])))
model.PACKYRS.linear.weight.data = torch.tensor(np.array(df.loc[df['Y.pred'] == 'DNAmPACKYRS', 'beta'][1:])).unsqueeze(0).float()
model.PACKYRS.linear.bias.data = torch.tensor(np.array(df.loc[df['Y.pred'] == 'DNAmPACKYRS', 'beta'].iloc[0])).float()
model.features_PACKYRS = indices = torch.tensor([all_features.index(item) for item in np.array(df.loc[df['Y.pred'] == 'DNAmPACKYRS', 'var']) if item in all_features]).long()

model.ADM = pya.models.LinearModel(input_dim=len(np.array(df.loc[df['Y.pred'] == 'DNAmadm'])))
model.ADM.linear.weight.data = torch.tensor(np.array(df.loc[df['Y.pred'] == 'DNAmadm', 'beta'][1:])).unsqueeze(0).float()
model.ADM.linear.bias.data = torch.tensor(np.array(df.loc[df['Y.pred'] == 'DNAmadm', 'beta'].iloc[0])).float()
model.features_ADM = indices = torch.tensor([all_features.index(item) for item in np.array(df.loc[df['Y.pred'] == 'DNAmadm', 'var']) if item in all_features]).long()

model.B2M = pya.models.LinearModel(input_dim=len(np.array(df.loc[df['Y.pred'] == 'DNAmB2M'])))
model.B2M.linear.weight.data = torch.tensor(np.array(df.loc[df['Y.pred'] == 'DNAmB2M', 'beta'][1:])).unsqueeze(0).float()
model.B2M.linear.bias.data = torch.tensor(np.array(df.loc[df['Y.pred'] == 'DNAmB2M', 'beta'].iloc[0])).float()
model.features_B2M = indices = torch.tensor([all_features.index(item) for item in np.array(df.loc[df['Y.pred'] == 'DNAmB2M', 'var']) if item in all_features]).long()

model.CystatinC = pya.models.LinearModel(input_dim=len(np.array(df.loc[df['Y.pred'] == 'DNAmCystatin_C'])))
model.CystatinC.linear.weight.data = torch.tensor(np.array(df.loc[df['Y.pred'] == 'DNAmCystatin_C', 'beta'][1:])).unsqueeze(0).float()
model.CystatinC.linear.bias.data = torch.tensor(np.array(df.loc[df['Y.pred'] == 'DNAmCystatin_C', 'beta'].iloc[0])).float()
model.features_CystatinC = indices = torch.tensor([all_features.index(item) for item in np.array(df.loc[df['Y.pred'] == 'DNAmCystatin_C', 'var']) if item in all_features]).long()

model.GDF15 = pya.models.LinearModel(input_dim=len(np.array(df.loc[df['Y.pred'] == 'DNAmGDF_15'])))
model.GDF15.linear.weight.data = torch.tensor(np.array(df.loc[df['Y.pred'] == 'DNAmGDF_15', 'beta'][1:])).unsqueeze(0).float()
model.GDF15.linear.bias.data = torch.tensor(np.array(df.loc[df['Y.pred'] == 'DNAmGDF_15', 'beta'].iloc[0])).float()
model.features_GDF15 = indices = torch.tensor([all_features.index(item) for item in np.array(df.loc[df['Y.pred'] == 'DNAmGDF_15', 'var']) if item in all_features]).long()

model.Leptin = pya.models.LinearModel(input_dim=len(np.array(df.loc[df['Y.pred'] == 'DNAmleptin'])))
model.Leptin.linear.weight.data = torch.tensor(np.array(df.loc[df['Y.pred'] == 'DNAmleptin', 'beta'][1:])).unsqueeze(0).float()
model.Leptin.linear.bias.data = torch.tensor(np.array(df.loc[df['Y.pred'] == 'DNAmleptin', 'beta'].iloc[0])).float()
model.features_Leptin = indices = torch.tensor([all_features.index(item) for item in np.array(df.loc[df['Y.pred'] == 'DNAmleptin', 'var']) if item in all_features]).long()

model.PAI1 = pya.models.LinearModel(input_dim=len(np.array(df.loc[df['Y.pred'] == 'DNAmpai_1'])))
model.PAI1.linear.weight.data = torch.tensor(np.array(df.loc[df['Y.pred'] == 'DNAmpai_1', 'beta'][1:])).unsqueeze(0).float()
model.PAI1.linear.bias.data = torch.tensor(np.array(df.loc[df['Y.pred'] == 'DNAmpai_1', 'beta'].iloc[0])).float()
model.features_PAI1 = indices = torch.tensor([all_features.index(item) for item in np.array(df.loc[df['Y.pred'] == 'DNAmpai_1', 'var']) if item in all_features]).long()

model.TIMP1 = pya.models.LinearModel(input_dim=len(np.array(df.loc[df['Y.pred'] == 'DNAmTIMP_1'])))
model.TIMP1.linear.weight.data = torch.tensor(np.array(df.loc[df['Y.pred'] == 'DNAmTIMP_1', 'beta'][1:])).unsqueeze(0).float()
model.TIMP1.linear.bias.data = torch.tensor(np.array(df.loc[df['Y.pred'] == 'DNAmTIMP_1', 'beta'].iloc[0])).float()
model.features_TIMP1 = indices = torch.tensor([all_features.index(item) for item in np.array(df.loc[df['Y.pred'] == 'DNAmTIMP_1', 'var']) if item in all_features]).long()

#### Linear model

In [8]:
grimage_weights = [
    0.000348777412272004,
    4.59105969389204e-07,
    3.49816671441537e-06,
    0.000143661105491888,
    0.00790270975255529,
    2.55560382039825e-05,
    -7.32066983502079e-06,
    0.0303981613409142,
    0.0300823182194075,
    -0.228468475622039
]

In [9]:
base_model = pya.models.LinearModel(input_dim=len(grimage_weights))

base_model.linear.weight.data = torch.tensor(grimage_weights).unsqueeze(0).float()
base_model.linear.bias.data = torch.tensor([0]).float()

model.base_model = base_model

## Load reference values

In [10]:
reference_df = pd.read_csv('datMiniAnnotation3_Gold.csv', index_col=0)
model.reference_values = reference_df.loc[model.features[:-2]]['gold'].tolist() + [1, 65] # 65 yo F

## Load preprocess and postprocess objects

In [11]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [12]:
model.postprocess_name = 'cox_to_years'
model.postprocess_dependencies = None

## Check all clock parameters

In [13]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Lu, Ake T., et al. "DNA methylation GrimAge strongly predicts '
             'lifespan and healthspan." Aging (albany NY) 11.2 (2019): 303.',
 'clock_name': 'grimage',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.18632/aging.101684',
 'notes': None,
 'research_only': True,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2019}
reference_values: [0.422480272528644, 0.935109546405548, 0.0162959729801047, 0.502691053893618, 0.910839576323153, 0.710155040209873, 0.479121329208521, 0.905888314944049, 0.279992670790348, 0.117900358329507, 0.940987438881091, 0.761621096809391, 0.0721244934513398, 0.0851830172952001, 0.222068390557704, 0.103705423432714, 0.91516014793103, 0.748331163695382, 0.903928589429489, 0.524090323888757, 0.894685558616447, 0.647988638853782, 0.0581747999131966, 0.830024180811995

## Basic test

In [14]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float).double()
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[ -8.6124],
        [ 66.5366],
        [ 49.7812],
        [-51.8447],
        [ 89.6747],
        [-65.7946],
        [ 89.6295],
        [-60.4899],
        [ 69.0053],
        [-58.3423]], dtype=torch.float64, grad_fn=<AddBackward0>)

## Save torch model

In [15]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [16]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: datMiniAnnotation3_Gold.csv
Deleted file: ElasticNet_DNAmProtein_Vars_model4.csv
